In [5]:
import pandas as pd
import numpy as np
import glob

file_path = 'data/*.xlsx'

files = glob.glob(file_path)
data = [
    [(sheet, pd.ExcelFile(file).parse(sheet)) 
     for sheet in pd.ExcelFile(file).sheet_names
    ] 
    for file in files
]

In [133]:
class IJ:
    def __init__ (self, data):
        self.data = dict(data)
        self.sheet_names, self.sheets = zip(*data)
        self.selected = 0
        
    def __iter__ (self):
        return iter(self.sheet_names)

    def select(self, sheet):
        for i in range(len(self.sheet_names)):
            if (self.sheet_names[i] == sheet):
                self.selected = i
                return
        raise Exception("No sheet by the name " + sheet + " sheet names are: ", self.sheet_names)
        
    def selecti(self, sheet):
        if (sheet >= len(self.sheet_names)):
            raise Exception("Index ", sheet, " out of range for size ", len(self.sheet_names))
        else:
            self.selected = sheet
            
    def select_next(self, i=1):
        self.selected = (self.selected + i) % len(self.sheet_names)
    
    def geti(self, indexes, i=None):
        return self.get(i).iloc[:, indexes]
        
    def gett(self, tags, i=None):
        return self.get(i)[tags]
        
    def get(self, i=None):
        return self.sheets[self.selected if i is None else i]

data2 = [IJ(d) for d in data]